In [2]:
import pandas as pd
import geopandas as gpd
import os

In [3]:
#path to read

path_to_alt='/home/angelica/Documents/py_linux/pylocss/data/karina/ourlakes/tmb_lakes/'

path_to_ts="/home/angelica/Documents/py_linux/pylocss/data/karina/ourlakes/ts_lakes/"
# open lakes
path_lakes='/home/angelica/Dropbox/Research/LOCSS/other_lakes/Selected_lakes/all_lakes_paper.shp'
df_sel_lakes=gpd.read_file(path_lakes)
# df_sel_lakes.loc[df_sel_lakes['lake_id']=='9129']#.unique()

In [4]:
time_fig='20230912_10_39'
df_final_altis=pd.read_csv('../data/results/preprossed_altis_data_'+time_fig+'.csv', sep=',',low_memory=False, parse_dates=['date'])

In [21]:
df_final_altis.loc[df_final_altis['lake_id']==19126]#.unique()#.orbit.unique()#head()#['decimal_y'].round(3)


,Unnamed: 0,decimal_y,height,nobs_altis,year,month,day,hour,date,name_altis,...,n_int_d_10,n_days_int_d_10,int_height,int_corr_ts1ts2,int_rmsd_ts2,lake_name,source,lake_id,orbit,for_paper
2165,0,2019.206897,127.274074,5.0,2019,3,17,12,2019-03-17 12:00:00+00:00,AlTiS_TimeSeries_Sentinel-3A_0560_S0953,...,1.0,0.375,137.634306,0.566222,0.131315,UHE XingÃ³,RVBR,19126,280,Y
2166,1,2019.280871,127.086990,4.0,2019,4,13,12,2019-04-13 12:00:00+00:00,AlTiS_TimeSeries_Sentinel-3A_0560_S0953,...,1.0,0.375,137.360000,0.566222,0.131315,UHE XingÃ³,RVBR,19126,280,Y
2167,2,2019.354844,127.038751,5.0,2019,5,10,12,2019-05-10 12:00:00+00:00,AlTiS_TimeSeries_Sentinel-3A_0560_S0953,...,1.0,0.375,137.341792,0.566222,0.131315,UHE XingÃ³,RVBR,19126,280,Y
2168,3,2019.428817,127.019982,7.0,2019,6,6,12,2019-06-06 12:00:00+00:00,AlTiS_TimeSeries_Sentinel-3A_0560_S0953,...,1.0,0.375,137.364278,0.566222,0.131315,UHE XingÃ³,RVBR,19126,280,Y
2169,4,2019.502789,127.517895,7.0,2019,7,3,12,2019-07-03 12:00:00+00:00,AlTiS_TimeSeries_Sentinel-3A_0560_S0953,...,1.0,0.375,137.462139,0.566222,0.131315,UHE XingÃ³,RVBR,19126,280,Y
2170,5,2019.576762,127.236911,5.0,2019,7,30,12,2019-07-30 12:00:00+00:00,AlTiS_TimeSeries_Sentinel-3A_0560_S0953,...,1.0,0.375,137.523583,0.566222,0.131315,UHE XingÃ³,RVBR,19126,280,Y
2171,6,2019.650732,126.987858,7.0,2019,8,26,12,2019-08-26 12:00:00+00:00,AlTiS_TimeSeries_Sentinel-3A_0560_S0953,...,1.0,0.375,137.364306,0.566222,0.131315,UHE XingÃ³,RVBR,19126,280,Y
2172,7,2019.724707,127.151356,5.0,2019,9,22,12,2019-09-22 12:00:00+00:00,AlTiS_TimeSeries_Sentinel-3A_0560_S0953,...,1.0,0.375,137.430403,0.566222,0.131315,UHE XingÃ³,RVBR,19126,280,Y
2173,8,2019.798680,127.078261,5.0,2019,10,19,12,2019-10-19 12:00:00+00:00,AlTiS_TimeSeries_Sentinel-3A_0560_S0953,...,1.0,0.375,137.476069,0.566222,0.131315,UHE XingÃ³,RVBR,19126,280,Y
2174,9,2019.872652,127.138855,4.0,2019,11,15,12,2019-11-15 12:00:00+00:00,AlTiS_TimeSeries_Sentinel-3A_0560_S0953,...,1.0,0.375,137.466069,0.566222,0.131315,UHE XingÃ³,RVBR,19126,280,Y


In [5]:
#read data from Karinas results
col_id_fd='lake_id'
list_lakes=df_sel_lakes.loc[df_sel_lakes['source'].isin(['RVBR', 'LOCSS', 'ARNH', 'USGS']),col_id_fd].unique()
#TODO validate empty list
df_files=pd.DataFrame()
sep=' '

for folder in os.listdir(path_to_alt):
        path=path_to_alt+folder+'/'
        files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))&(f.endswith('ocog.dat'))]
        for f in files:
            f_name=path+f
            df_file=pd.read_csv(f_name, sep=sep)
            df_file['sat']=f[:3]
            df_file['orbit']=f[4:7]
            df_file['lake_id']=folder
            df_files=pd.concat([df_files, df_file], axis=0)
         

In [51]:
# df_files.loc[df_files['lake_id']=='19060', 'orbit'].unique()#.unique()
def aggregage_values_by_group(df, group_by_fds, sum_dict):
    
    df = df.groupby(group_by_fds).aggregate(sum_dict)
    df.columns = ['_'.join([str(c) for c in col]).strip() for col in df.columns.values]
    if isinstance(df, (pd.DatetimeIndex, pd.MultiIndex)):
        df = df.to_frame(index=False)
    return df.reset_index()

def change_df_col_names(df, new_colnames):
    '''Change all the names of a dataframe by the names in new_colnames. The columns are changed in the same order of new_colnames'''
    for nn in new_colnames:
        df.rename(columns={df.columns[new_colnames.index(nn)]:nn}, inplace=True) 
    return df

,time_k,track_k,height_k,lat_k,lon_k,OCval_k,cycle_k,sat_k,orbit_k,lake_id_k
0,2018.421,1,604.695288,-25.935925,-52.068620,84,8,S3B,038,19060
1,2018.421,1,604.253522,-25.944901,-52.070938,85,8,S3B,038,19060
2,2018.495,2,605.295264,-25.879445,-52.134856,82,10,S3B,038,19060
3,2018.495,2,604.319304,-25.891415,-52.137942,83,10,S3B,038,19060
4,2018.495,2,604.542770,-25.894407,-52.138714,82,10,S3B,038,19060
...,...,...,...,...,...,...,...,...,...,...
685,2023.596,97,606.628584,-25.805824,-52.115245,83,102,S3A,038,19060
686,2023.596,97,606.856427,-25.808736,-52.115995,82,102,S3A,038,19060
687,2023.596,97,606.418739,-25.878627,-52.133995,82,102,S3A,038,19060
688,2023.596,97,744.795534,-25.893186,-52.137747,95,102,S3A,038,19060


In [52]:
#for each lake in locss lakes list_lakes
# get the same orbit in both
# get the same time in both
#plot
for lake in list_lakes:
    df_lake=df_final_altis.loc[df_final_altis['lake_id']==int(lake)]
    orbits=df_lake['orbit'].unique()
    print(orbits, lake)
    for ob in orbits:
        df_lake_k=df_files.loc[(pd.to_numeric(df_files['orbit'])==ob)&(df_files['lake_id']==lake)]
        
        if not df_lake_k.empty:
            # print(df_lake_k)
            #rename columns in karinas file so it's not the same
            df_lake_k.columns=[i+'_k' for i in df_lake_k.columns]
            
            df_lake_s=df_lake.loc[df_lake['orbit']==ob]
            df_lake_s=df_lake_s.round({'decimal_y':3})
            
            df_lake_k_gb=df_lake_k.groupby(['time_k']).agg({'height_k':['median','count'],
                                                            'sat_k':'first',
                                                            'orbit_k':'first',
                                                            'lake_id_k':'first'})
            #merge here 
            # df_lake_merged=pd.merge(df_lake_k, df_lake_s, left_on=['time_k'], right_on=['decimal_y'], how='inner')
            df_lake_k_gb.to_csv('../data/temp_merged.csv')
            print(df_lake_s.shape)
            break
    # print(df_final_altis.loc[df_final_altis['lake_id']==int(lake)])
    break

[ 6 38] 19060
(52, 299)


In [ ]:
import pandas as pd
import os
# import geopandas as gpd
#test reading Karina's observations
#Pending to pass the code to full observations 


col_id_fd='lake_id'
# list_lakes=df_sel_lakes.loc[df_sel_lakes['source'].isin(['RVBR', 'LOCSS', 'ARNH', 'USGS']),col_id_fd].unique()
#TODO validate empty list
df_files_wlts=pd.DataFrame()
df_files_tfew=pd.DataFrame()
df_files_conv=pd.DataFrame()
sep=' '

for folder in os.listdir(path_to_ts):
        path=path_to_ts+folder+'/'
        files_wlts = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))&f.startswith('wlts_')&(f.endswith('ocog.dat'))]
        files_conv = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))&f.startswith('conv_')&(f.endswith('ocog.dat'))]
        files_tfew = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))&f.startswith('tfew_')&(f.endswith('ocog.dat'))]
        
        sep=' '
        # Files with model reconstruction time series
        for f in files_wlts:
            f_name=path+f
            df_file=pd.read_csv(f_name, sep=sep)
            df_file['ts_type']=f[:4]
            df_file['sat']=f[5:8]
            df_file['orbit']=f[9:12]
            df_file['retracker']=f[13:17]
            df_file['lake_id']=folder
            df_file=df_file.rename(columns={'time\t':'time','wl\t':'wl'})
            df_files_wlts=pd.concat([df_files_wlts, df_file], axis=0)
            # print(df_files_wlts)
            # break
        sep=','
        # Files with results of modeling 
        for f in files_conv:
            f_name=path+f
            df_file=pd.read_csv(f_name, sep=sep)
            df_file['ts_type']=f[:4]
            df_file['sat']=f[5:8]
            df_file['orbit']=f[9:12]
            df_file['retracker']=f[13:17]
            df_file['lake_id']=folder
            df_files_conv=pd.concat([df_files_conv, df_file], axis=0)
            
        for f in files_tfew:
            f_name=path+f
            if os.stat(f_name).st_size != 0: 
                df_file=pd.read_csv(f_name)
                print('Not Empty', df_file)



In [ ]:
#Verify the error type Karina mentioned
#Read ground observations
#Create the match with the altimeter series
#Summarize error
#Compare with our approach
